In [1]:
// To see less warnings
import org.apache.log4j._
Logger.getLogger("org").setLevel(Level.ERROR)

Intitializing Scala interpreter ...

Spark Web UI available at http://74e4be6e0134:4041
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1651728432589)
SparkSession available as 'spark'


import org.apache.log4j._


In [2]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}

import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}


In [3]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.regression.LinearRegression

val spark = SparkSession.builder()
                        .appName("LinearRegEval")
                        .config("spark.master", "local")
                        .getOrCreate()

import spark.implicits._

import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.regression.LinearRegression
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4b861c4a
import spark.implicits._


In [11]:
// Prepare training and test data.
val data = spark.read.option("header","true").option("inferSchema","true").format("csv").load("USA_Housing.csv")



// This will allow us to join multiple feature columns
// into a single column of an array of feautre values
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors


// Grab only numerical columns from the data
var df = data.select($"Price", $"Avg Area Income", $"Avg Area House Age", $"Avg Area Number of Rooms", $"Area Population")
// Convert to double 
df = df.withColumn("Avg Area Income", df("Avg Area Income").cast("double"))
df = df.withColumn("Avg Area House Age", df("Avg Area Income").cast("double"))
df = df.withColumn("Avg Area Number of Rooms", df("Avg Area Number of Rooms").cast("double"))
df = df.withColumn("Area Population", df("Area Population").cast("double"))

df = df.withColumn("Price", df("Price").cast("double"))

df = df.withColumnRenamed("Price", "label")

data: org.apache.spark.sql.DataFrame = [Avg Area Income: double, Avg Area House Age: double ... 4 more fields]
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
df: org.apache.spark.sql.DataFrame = [label: double, Avg Area Income: double ... 3 more fields]
df: org.apache.spark.sql.DataFrame = [label: double, Avg Area Income: double ... 3 more fields]
df: org.apache.spark.sql.DataFrame = [label: double, Avg Area Income: double ... 3 more fields]
df: org.apache.spark.sql.DataFrame = [label: double, Avg Area Income: double ... 3 more fields]
df: org.apache.spark.sql.DataFrame = [label: double, Avg Area Income: double ... 3 more fields]
df: org.apache.spark.sql.DataFrame = [label: double, Avg Area Income: double ... 3 more fields]
df: org.apache.sp...


In [17]:
// An assembler converts the input values to a vector
// A vector is what the ML algorithm reads to train a model

// Set the input columns from which we are supposed to read the values
// Set the name of the column where the vector will be stored
val assembler = new VectorAssembler().setInputCols(Array("Avg Area Income","Avg Area House Age","Avg Area Number of Rooms","Area Population")).setOutputCol("features")

// Use the assembler to transform our DataFrame to the two columns
val df1 = assembler.transform(df).select($"label", $"features")

assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_f6915188549b, handleInvalid=error, numInputCols=4
df1: org.apache.spark.sql.DataFrame = [label: double, features: vector]


In [18]:
val Array(training, test) = df1.randomSplit(Array(0.9, 0.1), seed = 12345)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]


In [19]:
training.show(4)

+-----------+--------------------+
|      label|            features|
+-----------+--------------------+
|15938.65792|[47320.65721,4732...|
|31140.51762|[37971.20757,3797...|
|88591.77016|[60167.67261,6016...|
|143027.3645|[35963.33081,3596...|
+-----------+--------------------+
only showing top 4 rows



In [20]:
test.show(4)

+-----------+--------------------+
|      label|            features|
+-----------+--------------------+
|311111.2006|[59801.49103,5980...|
|313651.5032|[59141.79644,5914...|
|365929.5973|[58342.36779,5834...|
|398909.5099|[50847.11311,5084...|
+-----------+--------------------+
only showing top 4 rows



In [21]:
// param grid


val lr = new LinearRegression()

// We use a ParamGridBuilder to construct a grid of parameters to search over.
// TrainValidationSplit will try all combinations of values and determine best model using
// the evaluator.
val paramGrid = new ParamGridBuilder().addGrid(lr.regParam, Array(0.1, 0.01)).addGrid(lr.fitIntercept).addGrid(lr.elasticNetParam, Array(0.0, 0.5, 1.0)).build()

// In this case the estimator is simply the linear regression.
// A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
// 80% of the data will be used for training and the remaining 20% for validation.
val trainValidationSplit = new TrainValidationSplit().setEstimator(lr).setEvaluator(new RegressionEvaluator).setEstimatorParamMaps(paramGrid).setTrainRatio(0.8)


// Run train validation split, and choose the best set of parameters.
val model = trainValidationSplit.fit(training)

// Make predictions on test data. model is the model with combination of parameters
// that performed best.
model.transform(test).select("features", "label", "prediction").show()



+--------------------+-----------+------------------+
|            features|      label|        prediction|
+--------------------+-----------+------------------+
|[59801.49103,5980...|311111.2006| 754922.0464481027|
|[59141.79644,5914...|313651.5032| 587039.9686413496|
|[58342.36779,5834...|365929.5973| 576055.3632130935|
|[50847.11311,5084...|398909.5099| 628280.4269229695|
|[52202.85372,5220...|404976.3659| 654193.7257026793|
|[41007.45867,4100...|494742.5436| 568822.9041552809|
|[44806.00516,4480...|497579.4466| 759559.0305158193|
|[44247.14904,4424...|503065.5056|393037.18210031977|
|[45353.13038,4535...|544320.6881| 739439.1460034964|
|[51510.69878,5151...|549167.9399| 755447.4889441873|
|[67186.26237,6718...|554702.6802| 588802.7504048888|
|[44328.2563,44328...|601007.3512| 455471.2135610604|
|[59777.37269,5977...|614700.7371| 910251.8477743613|
|[57925.0447,57925...|633875.9302| 1115395.441577265|
|[57576.05395,5757...|634218.4408| 822872.4896821738|
|[52022.15897,5202...|642855

lr: org.apache.spark.ml.regression.LinearRegression = linReg_cec1b43155fa
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	linReg_cec1b43155fa-elasticNetParam: 0.0,
	linReg_cec1b43155fa-fitIntercept: true,
	linReg_cec1b43155fa-regParam: 0.1
}, {
	linReg_cec1b43155fa-elasticNetParam: 0.0,
	linReg_cec1b43155fa-fitIntercept: true,
	linReg_cec1b43155fa-regParam: 0.01
}, {
	linReg_cec1b43155fa-elasticNetParam: 0.0,
	linReg_cec1b43155fa-fitIntercept: false,
	linReg_cec1b43155fa-regParam: 0.1
}, {
	linReg_cec1b43155fa-elasticNetParam: 0.0,
	linReg_cec1b43155fa-fitIntercept: false,
	linReg_cec1b43155fa-regParam: 0.01
}, {
	linReg_cec1b43155fa-elasticNetParam: 0.5,
	linReg_cec1b43155fa-fitIntercept: true,
	linReg_cec1b43155fa-regParam: 0.1
}, {
	linReg_cec1b43155fa-elasticNetParam...
